
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>


# 4 Lab - Create a Pipeline  
### Estimated Duration: ~15-20 minutes

In this lab, you'll migrate a traditional ETL workflow to a pipeline for incremental data processing. You'll practice building streaming tables and materialized views using Lakeflow Spark Declarative Pipelines syntax.

#### Your Tasks:
- Create a new Pipeline  
- Convert traditional SQL ETL to declarative syntax for incremental processing 
- Configure pipeline settings  
- Define data quality expectations  
- Validate and run the pipeline

### Learning Objectives

By the end of this lesson, you will be able to:
- Create a pipeline and execute it successfully using the new Lakeflow Pipeline Editor.
- Modify and configure pipeline settings to align with specific data processing requirements.
- Integrate data quality expectations into a pipeline and evaluate their effectiveness.

## REQUIRED - SELECT CLASSIC COMPUTE (your cluster starts with **labuser**)

Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default.

Follow these steps to select the classic compute cluster:

1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

1. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

    - In the drop-down, select **More**.

    - In the **Attach to an existing compute resource** pop-up, select the first drop-down. You will see a unique cluster name in that drop-down. Please select that cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

1. Find the triangle icon to the right of your compute cluster name and click it.

1. Wait a few minutes for the cluster to start.

1. Once the cluster is running, complete the steps above to select your cluster.

## A. Classroom Setup

Run the following cell to configure your working environment for this course.

**NOTE:** The `DA` object is only used in Databricks Academy courses and is not available outside of these courses. It will dynamically create and reference the information needed to run the course.

In [0]:
%run ./Includes/Classroom-Setup-4

## B. SCENARIO

Your data engineering team has identified an opportunity to modernize an existing ETL pipeline that was originally developed in a Databricks notebook. While the current pipeline gets the job done, it lacks the scalability, observability, efficiency and automated data quality features required as your data volume and complexity grow.

To address this, you've been asked to migrate the existing pipeline to a Lakeflow Spark Declarative Pipeline. Spark Declarative Pipelines will enable your team to define data transformations more declaratively, apply data quality rules, and benefit from built-in optimization, lineage tracking and monitoring.

Your goal is to refactor the original notebook based logic (shown in the cells below) into a Spark Declarative Pipeline.

### REQUIREMENTS:
  - Migrate the ETL code below to a Spark Declarative Pipeline.
  - Add the required data quality expectations to the bronze table and silver table:
  - Create materialized views for the most up to date aggregated information.

Follow the steps below to complete your task.

### B1. Explore the Raw Data

1. Complete the following steps to view where our lab's streaming raw source files are coming from:

   a. Select the **Catalog** icon ![Catalog Icon](./Includes/images/catalog_icon.png) in the left navigation bar.  
   
   b. Expand your **labuser** catalog.  
   
   c. Expand the **default** schema.  
   
   d. Expand **Volumes**.  
   
   e. Expand the **lab_files** volume.  
   
   f. You should see a single CSV file named **employees_1.csv**. If not, refresh the catalogs.  
   
   g. The files in the **lab_files** volume will be the data source files you will be ingesting.

2. Run the cell below to view the raw CSV file in your **lab_files** volume. Notice the following:

   - It’s a simple CSV file separated by commas.  
   - It contains headers.  
   - It has 7 rows in total (6 data records and 1 header row).  
   - The first record (row 2) is a test record and should not be included in the pipeline and will be dropped by a data quality expectation.

In [0]:
spark.sql(f'''
        SELECT *
        FROM csv.`/Volumes/{DA.catalog_name}/default/lab_files/`
        ''').display()

### B2. Current ETL Code

Run each cell below to view the results of the current ETL pipeline. This will give you an idea of the expected output. Don’t worry too much about the data transformations within the SQL queries.

The focus of this lab is on using **declarative SQL** and creating a **Spark Declarative Pipeline**. You will not need to modify the transformation logic, only the `CREATE` statements and `FROM` clauses to ensure data is read and processed incrementally in your pipeline.

#### B2.1 - CSV to Bronze

Explore the code and run the cell. Observe the results. Notice that:

- The CSV file in the volume is read in as a table named **employees_bronze_lab4** in the **labuser.lab_1_bronze_db** schema.  
- The table contains 6 rows with the correct column names.

Think about what you will need to change when migrating this to a Spake Declarative Pipeline. Hints are added as comments in the code below.

**NOTE:** In your Spark Declarative Pipeline we will want to add data quality expectations to document any bad data coming into the pipeline.

In [0]:
%sql
-- Specify to use your labuser catalog from the course DA object
USE CATALOG IDENTIFIER(DA.catalog_name);


CREATE OR REPLACE TABLE lab_1_bronze_db.employees_bronze_lab4  -- You will have to modify this to create a streaming table in the pipeline
AS
SELECT 
  *,
  current_timestamp() AS ingestion_time,
  _metadata.file_name as raw_file_name
FROM read_files(                                           -- You will have to modify FROM clause to incrementally read in data
  '/Volumes/' || DA.catalog_name || '/default/lab_files',  -- You will have to modify this path in the pipeline to your specific raw data source
  format => 'CSV',
  header => 'true'
);


-- Display table
SELECT *
FROM lab_1_bronze_db.employees_bronze_lab4;

#### B2.2 - Bronze to Silver

Run the cell below to create the table **labuser.lab_2_silver_db.employees_silver_lab4** and explore the results. Notice that a few simple data transformations were applied to the bronze table, and metadata columns were removed.

Think about what you will need to change when migrating this to a Spark Declarative Pipeline. Hints are added as comments in the code below.

**NOTE:** For simplicity, we are leaving the **test** row in place, and you will remove it using data quality expectations. Typically, we could have just filtered out the null value(s).

In [0]:
%sql
CREATE OR REPLACE TABLE lab_2_silver_db.employees_silver_lab4 -- You will have to modify this to create a streaming table in the pipeline
AS
SELECT
  EmployeeID,
  FirstName,
  upper(Country) AS Country,
  Department,
  Salary,
  HireDate,
  date_format(HireDate, 'MMMM') AS HireMonthName,
  year(HireDate) AS HireYear, 
  Operation
FROM lab_1_bronze_db.employees_bronze_lab4;                    -- You will have to modify FROM clause to incrementally read in data


-- Display table
SELECT *
FROM lab_2_silver_db.employees_silver_lab4;

#### B2.3 - Silver to Gold
The code below creates two traditional views to aggregate the silver tables.

1. Run the cell to create a view that calculates the total number of employees and total salary by country.

    Think about what you will need to change when migrating this to a Spark Declarative Pipeline. A hint is added as a comment in the code below.

In [0]:
%sql
CREATE OR REPLACE VIEW lab_3_gold_db.employees_by_country_gold_lab4 -- You will have to modify this to create a materialized view in the pipeline
AS
SELECT 
  Country,
  count(*) AS TotalEmployees,
  sum(Salary) AS TotalSalary
FROM lab_2_silver_db.employees_silver_lab4
GROUP BY Country;


-- Display view
SELECT *
FROM lab_3_gold_db.employees_by_country_gold_lab4;

2. Run the cell to create a view that calculates the salary by department.

    Think about what you will need to change when migrating this to a Spark Declarative Pipeline. A hint is added as a comment in the code below.

In [0]:
%sql
CREATE OR REPLACE VIEW lab_3_gold_db.salary_by_department_gold_lab4  -- You will have to modify this to create a materialized view in the pipeline
AS
SELECT
  Department,
  sum(Salary) AS TotalSalary
FROM lab_2_silver_db.employees_silver_lab4
GROUP BY Department;


-- Display view
SELECT *
FROM lab_3_gold_db.salary_by_department_gold_lab4;

#### B2.4 - Delete the Tables

Run the cell below to delete all the tables you created above. You will recreate them as streaming tables and materialized views in the Spark Declarative Pipeline.

**NOTE:** If you have created the streaming tables and materialized views with Spark Declarative Pipelines and want to drop them to redo this lab, the following code will not work with the lab's default **No isolation shared cluster**. You will have to run the cells in this notebook using Serverless or manually delete the pipeline and tables.

In [0]:
%sql
DROP TABLE IF EXISTS lab_1_bronze_db.employees_bronze_lab4;
DROP TABLE IF EXISTS lab_2_silver_db.employees_silver_lab4;
DROP VIEW IF EXISTS lab_3_gold_db.employees_by_country_gold_lab4;
DROP VIEW IF EXISTS lab_3_gold_db.salary_by_department_gold_lab4;

Run the cell below to view and copy the path to your **lab_files** volume. You will need this path when building your pipeline to reference your data source files.

**NOTE:** You can also navigate to the volume and copy the path using the UI.

In [0]:
print(f'/Volumes/{DA.catalog_name}/default/lab_files')

## C. TO DO: Create the Lakeflow Spark Declarative Pipeline (Steps)

After you have explored the traditional ETL code to create the tables and views, it's time to modify that syntax to declarative SQL for your new pipeline.

You will have to complete the following:

**NOTE:** The solution files can be found in the **4 - Lab Solution Project**. All code is in the one **ingest.sql** file:

1. Create a Spark Declarative Pipeline and name it **Lab4 - firstname pipeline project**.

    - Select your **labuser** catalog  

    - Select the **default** schema  

    - Select the **Start with sample code in SQL** language  

    - **NOTE:** The Spark Declarative Pipeline will contain sample files and notebooks. You can exclude the sample files from the pipeline before you run the pipeline.


2. Migrate the ETL code (shown below for each step as markdown) into one or more files and folders to organize your pipeline (you can also put everything in a single file if you want).
<br></br>
##### 2a. Modify the code (shown below) to create the **bronze** streaming table by completing the following:

```SQL
CREATE OR REPLACE TABLE lab_1_bronze_db.employees_bronze_lab4  -- You will have to modify this to create a streaming table in the pipeline
AS
SELECT 
    *,
    current_timestamp() AS ingestion_time,
    _metadata.file_name as raw_file_name
FROM read_files(                                           -- You will have to modify FROM clause to incrementally read in data
    '/Volumes/' || DA.catalog_name || '/default/lab_files',  -- You will have to modify this path in the pipeline to your specific raw data source
    format => 'CSV',
    header => 'true'
);
```

- Modify the `CREATE OR REPLACE TABLE` statement to create a streaming table.  

- Add the keyword `STREAM` in the `FROM` clause to incrementally ingest data from the delta table.

- Modify the path in the `FROM` clause to point to your **labuser.default.lab_files** volume path (example: `/Volumes/labuser1234/default/lab_files`). You can statically add the path in the `read_files` function, or use a configuration parameter.
- **NOTE:** You can't use the `DA` object in your path. Remember to add a static path or configuration parameter.

<br></br>
##### 2b. Modify the code (shown below) to create the **silver** streaming table by completing the following in your pipeline project:

```
CREATE OR REPLACE TABLE lab_2_silver_db.employees_silver_lab4 -- You will have to modify this to create a streaming table in the pipeline
AS
SELECT
    EmployeeID,
    FirstName,
    upper(Country) AS Country,
    Department,
    Salary,
    HireDate,
    date_format(HireDate, 'MMMM') AS HireMonthName,
    year(HireDate) AS HireYear, 
    Operation
FROM lab_1_bronze_db.employees_bronze_lab4;                    -- You will have to modify FROM clause to incrementally read in data
```

- Modify the `CREATE OR REPLACE TABLE` statement to create a streaming table.  

- Add the keyword `STREAM` in the `FROM` clause to incrementally ingest data.

- Add the following data quality expectations:      
```
CONSTRAINT check_country EXPECT (Country IN ('US','GR')),
CONSTRAINT check_salary EXPECT (Salary > 0),
CONSTRAINT check_null_id EXPECT (EmployeeID IS NOT NULL) ON VIOLATION DROP ROW

```
<br></br>
##### 2c. Replace the `CREATE OR REPLACE VIEW` statement in the two views to create materialized views instead of traditional views in your pipeline project.

```
CREATE OR REPLACE VIEW lab_3_gold_db.employees_by_country_gold_lab4 -- You will have to modify this to create a materialized view in the pipeline
AS
SELECT 
    Country,
    count(*) AS TotalEmployees,
    sum(Salary) AS TotalSalary
FROM lab_2_silver_db.employees_silver_lab4
GROUP BY Country;


CREATE OR REPLACE VIEW lab_3_gold_db.salary_by_department_gold_lab4  -- You will have to modify this to create a materialized view in the pipeline
AS
SELECT
    Department,
    sum(Salary) AS TotalSalary
FROM lab_2_silver_db.employees_silver_lab4
GROUP BY Department;

```
<br></br>
3. Pipeline configuration requirements:

- Your Spark Declarative Pipeline should use **Serverless** compute.  

- Your pipeline should use your **labuser** catalog by default.  

- Your pipeline should use your **default** schema by default.

- Make sure your pipeline is including your .sql file only.

- (OPTIONAL) If using a configuration variable to point to your path make sure it is setup and applied in the `read_files` function.

<br></br>
4. When complete, run the pipeline. Troubleshoot any errors.

<br></br>

##### Final Spark Declarative Pipeline Image  
Below is what your final pipeline should look like after the first run with a single CSV file.

![Final Lab4 Pipeline](./Includes/images/lab4_solutiongraph.png)

### LAB SOLUTION (optional)
If you need the solution, you can view the lab solution code in the **4 - Lab Solution Project** folder. You can also execute the code below to automatically create the Spark Declarative Pipeline with the necessary configuration settings for your specific lab.

**NOTE:** After you run the cell, wait 30 seconds for the pipeline to finish creating. Then open one of the files in the **4 - Lab Solution Project** folder to open the new Spark Declarative Pipeline editor.

In [0]:
create_declarative_pipeline(pipeline_name=f'4 - Lab Solution Project - {DA.catalog_name}', 
                            root_path_folder_name='4 - Lab Solution Project',
                            catalog_name = DA.catalog_name,
                            schema_name = 'default',
                            source_folder_names=['solution'],
                            configuration = {'source':f'/Volumes/{DA.catalog_name}/default/lab_files'})

## D. Explore the Streaming Tables and Materialized Views

After you have created and run your Spark Declarative Pipeline, complete the following tasks to explore your new streaming tables and materialized views.

1. In the catalog explorer on the left, expand your **labuser** catalog and expand the following schemas:
   - **lab_1_bronze_db**
   - **lab_2_silver_db**
   - **lab_3_gold_db**

    You should see the two streaming tables and materialized views within your schemas (if you don't use the solution, you won't have the **_solution** at the end of the streaming tables and materialized views):

   <img src="./Includes/images/lab4_solution_schemas.png" alt="Objects in Schemas" width="350">


2. Run the cell below to view the data in your **labuser.lab_1_bronze_db.employees_bronze_lab4** streaming table. Notice that the first row contains a `null` **EmployeeID**.

**NOTE:** If you ran the solution pipeline, the streaming table is named **employees_bronze_lab4_solution**.

In [0]:
%sql
SELECT *
FROM lab_1_bronze_db.employees_bronze_lab4;

3. Run the cell below to view the data in your **labuser.lab_2_silver_db.employees_silver_lab4** streaming table. Notice that the silver table removed the **EmployeeID** value that contained a `null` using a data quality expectation.

**NOTE:** If you ran the solution pipeline, the streaming table is named **employees_silver_lab4_solution**.

In [0]:
%sql
SELECT *
FROM lab_2_silver_db.employees_silver_lab4;

4. Run the cell below to view the data in your **labuser.lab_3_gold_db.employees_by_country_gold_lab4** materialized view. 

    **Final Results**
    | Country | TotalCount | TotalSalary |
    |---------|------------|-------------|
    | GR      | 2          | 108000      |
    | US      | 3          | 201000      |

**NOTE:** If you ran the solution pipeline, the materialized view is named **employees_by_country_gold_lab4_solution**.

In [0]:
%sql
SELECT *
FROM lab_3_gold_db.employees_by_country_gold_lab4
ORDER BY TotalSalary;

5. Run the cell below to view the data in your **labuser.lab_3_gold_db.salary_by_department_gold_lab4** materialized view. 

    **Final Results**
    | Department  | TotalSalary |
    |-------------|-------------|
    | Sales          | 141000      |
    | IT          | 168000      |

**NOTE:** If you ran the solution pipeline, the materialized view is named **salary_by_department_gold_lab4_solution**.

In [0]:
%sql
SELECT *
FROM lab_3_gold_db.salary_by_department_gold_lab4
ORDER BY TotalSalary;

## E. CHALLENGE SCENARIO (OPTIONAL IN LIVE TEACH)
### Duration: ~10 minutes

**NOTE:** *If you finish early in a live class, feel free to complete the challenge below. The challenge is optional and most likely won't be completed during the live class. Only continue if your Spark Declarative Pipeline was set up correctly in the previous section by comparing your pipeline to the solution image.*

**SCENARIO:** In the challenge, you will land a new CSV file in your **lab_files** cloud storage volume and rerun the pipeline to watch the Spark Declarative Pipeline only ingest the new data.

1. Run the cell below to copy another file to your **labuser.default.lab_user** volume.


In [0]:
LabSetup.copy_file(copy_file = 'employees_2.csv', 
                   to_target_volume = f'/Volumes/{DA.catalog_name}/default/lab_files')

2. In the left navigation area, navigate to your **labuser.default.lab_files** volume and expand the volume. Confirm it contains two CSV files: 
    - **employees_1.csv** 
    - **employees_2.csv**

**NOTE:** You might have to refresh your catalogs if the file is not shown.

3. Run the cell below to preview only the new CSV file and view the results. Notice that the new CSV file contains employee information:

    - Contains 4 rows.  
    - The **Operation** column specifies an action for each employee (e.g., update the record, delete the record, or add a new employee).

**NOTE:** Don’t worry about the **Operation** column yet. We’ll cover how to capture these specific changes in your data (Change Data Capture) in a later demonstration.

In [0]:
%sql
SELECT *
FROM read_files(                                           
  '/Volumes/' || DA.catalog_name || '/default/lab_files/employees_2.csv',  
  format => 'CSV',
  header => 'true'
);

4. Now that you have explored the new CSV file in cloud storage, go back to your Spark Declarative Pipeline and select **Run pipeline**. Notice that the pipeline only read in the new file in cloud storage.


##### Final Spark Declarative Pipeline Image
Below is what your final pipeline should look like after the first run with a single CSV file.

![Final Challenge Lab4 DLT Pipeline](./Includes/images/lab4_solutionchallenge.png)

5. Explore the history of your streaming tables using the Catalog Explorer. Notice that there are two appends to both the **bronze** and **silver** tables.

&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>